In [1]:
import pandas as pd

In [2]:
#READ NORMALIZED DATA FOR TARGET VERSION OF PIPELINE
df = pd.read_csv("C:\\Users\\brush\\Projekt_NLP\\normalizedResult.tsv", sep="\t")
testdf = pd.read_csv("C:\\Users\\brush\\Projekt_NLP\\TestDaten\\Testdaten.tsv", usecols=['c_text'], sep="\t")

In [3]:
import spacy
nlp = spacy.load("de_core_news_sm")

texts=testdf['c_text']
tweet_words = []
n_text = ""
for tweet in texts:
    doc = nlp(tweet)
    for token in doc:
        if token.is_alpha:
            n_text+=token.text.lower()+" "
    
    tweet_words.append(n_text)
    n_text=""

In [4]:
counter=0
for t in tweet_words:
    tweet_words[counter] = t[:-1]
    counter+=1

In [5]:
testdf['c_text']=tweet_words

In [6]:
#CREATE TWO LISTS THAT CONTAIN EACH TEXT OF A TWEET AND ITS TARGET
df=df.dropna()
target=(df.iloc[0:8132 , 2 ].to_list())
text=df.iloc[0:8132, 1 ].to_list()

In [7]:
testdf=testdf.dropna()
testtext=testdf['c_text'].to_list()

In [8]:
#CREATE DATAFRAME THAT CONTAINS TWEET TEXT AND TARGET
df=pd.DataFrame ({'Text': text , 'Target': target })
testdf=pd.DataFrame ({'Text': testtext })

In [9]:
X_train = df.Text
y_train = df.Target
X_test = testdf.Text

In [10]:
###TARGET SPLITTING OF 0.8 DATA###
targetPerson=[]
targetGroup=[]
targetPublic=[]

X_trainList = X_train.tolist()
y_trainList = y_train.tolist()

counter=0
for t in X_trainList:
    if y_trainList[counter] == 0:
        targetPerson.append(t)
    elif y_trainList[counter] == 1:
        targetGroup.append(t)
    else:
        targetPublic.append(t)
    counter+=1
###TARGET SPLITTING END###

In [11]:
###UNIQUE WORD SPLITTING OF 0.8 DATA###
personWords=[]
groupWords=[]
publicWords=[]

for tweet in targetPerson:
    personWords.append(tweet.split())
personWords = sum(personWords, [])

for tweet in targetGroup:
    groupWords.append(tweet.split())
groupWords = sum(groupWords, [])

for tweet in targetPublic:
    publicWords.append(tweet.split())
publicWords = sum(publicWords, [])

personUnique=[]
groupUnique=[]
publicUnique=[]

for word in personWords:
    if word not in personUnique:
        personUnique.append(word)
        
for word in groupWords:
    if word not in groupUnique:
        groupUnique.append(word)
        
for word in publicWords:
    if word not in publicUnique:
        publicUnique.append(word)
        
onlyPerson=[]

for word in personUnique:
    if word not in groupUnique:
        if word not in publicUnique:
            onlyPerson.append(word)
            
onlyGroup=[]

for word in groupUnique:
    if word not in personUnique:
        if word not in publicUnique:
            onlyGroup.append(word)
            
onlyPublic=[]

for word in publicUnique:
    if word not in personUnique:
        if word not in groupUnique:
            onlyPublic.append(word)
###UNIQUE WORD SPLITTING END###

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification 
import xgboost as xgb

print(len(X_test))
#tfidf
#vektorisieren
vectorizer = TfidfVectorizer() 
Xtrain = vectorizer.fit_transform(X_train) 
Xtest = vectorizer.transform(X_test) 
#vektorisieren
print(len(X_test))

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

x = X_train.tolist()
Xtrain = model.encode(x) 
xt = X_test.tolist()
Xtest = model.encode(xt)

2034
2034


In [13]:
print(len(X_test))

2034


In [14]:
#create classifier and train model

clf = xgb.XGBClassifier(objective="binary:logistic", random_state=2021,
                           n_estimators=100,
                           gamma=0,
                           min_child_weight=1,
                           learning_rate=0.3,
                           reg_alpha=6,
                           max_depth=6,
                           colsample_bytree=0.7, colsample_bylevel=0.6, colsample_bynode=1
                       )

#2. fit with X_train and y_train
clf.fit(Xtrain, y_train)

#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(Xtest)

In [15]:
y_pred2=y_pred
personCount=0
groupCount=0
publicCount=0
counter=0
single_pronouns=["du", "dich", "dein", "er", "sein", "mein", "meine", "mir"]
single_adress=["herr", "herrn", "frau", "junge", "nazi"]
single_politicians=["merkel", "söder", "spahn", "soeder", "laschet", "baerbock", "bundeskanzlerin", "bundeskanzler", "kanzler", "kanzlerin", "putin", "trump", "erdogan"]
group_pronouns=["euch", "eure", "euer", "deren", "ihr", "unsere", "uns"]
group_politics=["spd", "cdu", "csu", "afd", "grünen", "grüne", "union", "linken", "nazis", "rechten", "partei", "land", "terroristen", "demokratie"]
group_religion=["muslime", "islamische", "islam", "islamistische", "christen", "juden"]
group_countries=["deutschland", "deutsche", "bürger", "araber", "ausländer", "schwarzen"]
public_pronouns=["alle", "wir", "menschen", "volk", "bevölkerung", "welt"]

In [16]:
#7. run analyzer for target correction
#7.1. filter out person tweets out of group tweets and correct prediction
for s in y_pred2:
    if y_pred2[counter] == 1:
        doc=X_test[counter].split()
        for token in doc:
            if token in onlyPerson:
                personCount+=1
            if token in single_pronouns or token in single_adress or token in single_politicians:
                personCount+=1
        if personCount >= 5:
            y_pred2[counter]=0
        personCount=0
    counter+=1
counter=0

#7.2. filter out group tweets out of person tweets and correct prediction
for s in y_pred2:
    if y_pred2[counter] == 0:
        doc=X_test[counter].split()
        for token in doc:
            if token in onlyGroup:
                groupCount+=1
            if token in group_pronouns or token in group_politics or token in group_countries or token in group_religion:
                groupCount+=1
        if groupCount >= 5:
            y_pred2[counter]=1
        groupCount=0
    counter+=1
counter=0

#7.3. filter out public tweets out of person and group tweets and correct prediction
personCount=0
groupCount=0
publicCount=0
for s in y_pred2:
    if y_pred2[counter] == 0 or y_pred2[counter] == 1:
        doc=X_test[counter].split()
        for token in doc:
            if token in onlyPublic:
                publicCount+=1
            if token in public_pronouns:
                publicCount+=1
            if token in onlyGroup:
                groupCount+=1
            if token in group_pronouns or token in group_politics or token in group_countries or token in group_religion:
                groupCount+=1
            if token in onlyPerson:
                personCount+=1
            if token in single_pronouns or token in single_adress or token in single_politicians:
                personCount+=1
        if publicCount > groupCount - 3 or publicCount > personCount - 3:
            y_pred2[counter]=2
        publicCount=0
    counter+=1
counter=0

In [21]:
testdfFinal=pd.DataFrame({'Klasse': y_pred2})
testdfFinal.to_csv('C:\\Users\\brush\\Projekt_NLP\\Result\\Hybrides Team Target_1b.csv', sep="\t")